In [1]:
#N件前のデータ取得方法

# customerごとにreserve_datetimeで並び替え
# groupby関数のあとにapply関数を適用することによって、groupごとに並び替える
# sort_values関数によってデータを並び替え、axisが0の場合は行、1の場合は列を並び替え
result = reserve_tb \
  .groupby('customer_id') \
  .apply(lambda group:
         group.sort_values(by='reserve_datetime', axis=0, inplace=False))

# resultはすでに、customer_idごとにgroup化されている
# customerごとに2つ前のtotal_priceをbefore_priceとして保存
# shift関数は、periodsの引数の数だけデータ行を下にずらす関数
result['before_price'] = \
  pd.Series(result['total_price'].shift(periods=2))
result

NameError: name 'reserve_tb' is not defined

In [4]:

import tkinter as tk
import tkinter.filedialog as fd
import csv
import pandas as pd
import csv

root = tk.Tk()
root.withdraw()

file = fd.askopenfilename(
    title = "Choose a file",
    filetypes=[("*", ".csv")]
)

with open(file, encoding="utf_8") as file:
    reader = pd.read_csv(file)

In [16]:
reserve_tb = reader
customer_tb = customer_id


In [17]:
#ｎ件前のデータ取得

# customerごとにreserve_datetimeで並び替え
# groupby関数のあとにapply関数を適用することによって、groupごとに並び替える
# sort_values関数によってデータを並び替え、axisが0の場合は行、1の場合は列を並び替え
result = reserve_tb \
  .groupby('customer_id') \
  .apply(lambda group:
         group.sort_values(by='reserve_datetime', axis=0, inplace=False))

# resultはすでに、customer_idごとにgroup化されている
# customerごとに2つ前のtotal_priceをbefore_priceとして保存
# shift関数は、periodsの引数の数だけデータ行を下にずらす関数
result['before_price'] = \
  pd.Series(result['total_price'].shift(periods=2))

In [18]:
result

reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                             
c_1         0            r1     h_75         c_1  2016-03-06 13:09:42   
            1            r2    h_219         c_1  2016-07-16 23:39:55   
            2            r3    h_179         c_1  2016-09-24 10:03:17   
            3            r4    h_214         c_1  2017-03-08 03:20:10   
            4            r5     h_16         c_1  2017-09-05 19:50:37   
...                     ...      ...         ...                  ...   
c_999       4023      r4024    h_244       c_999  2016-10-06 18:01:34   
            4024      r4025    h_160       c_999  2017-03-11 11:56:05   
            4025      r4026    h_129       c_999  2017-06-27 23:00:02   
            4026      r4027     h_97       c_999  2017-09-29 05:24:57   
            4027      r4028     h_27       c_999  2018-03-14 05:01:45   

                 checkin_date checkin_time checkout_date  people_num  \
customer_id                                                            
c_1         0      2016-03-26     10:00:00    2016-03-29           4   
            1      2016-07-20     11:30:00    2016-07-21           2   
            2      2016-10-19     09:00:00    2016-10-22           2   
            3      2017-03-29     11:00:00    2017-03-30           4   
            4      2017-09-22     10:30:00    2017-09-23           3   
...                       ...          ...           ...         ...   
c_999       4023   2016-10-07     11:30:00    2016-10-09           3   
            4024   2017-03-27     10:00:00    2017-03-30           1   
            4025   2017-07-10     09:30:00    2017-07-11           2   
            4026   2017-10-09     10:30:00    2017-10-10           2   
            4027   2018-04-02     11:30:00    2018-04-04           2   

                  total_price  before_price  
customer_id                                  
c_1         0           97200           NaN  
            1           20600           NaN  
            2           33600       97200.0  
            3          194400       20600.0  
            4           68100       33600.0  
...                       ...           ...  
c_999       4023        59400       96000.0  
            4024        37200       62400.0  
            4025        16000       59400.0  
            4026        41800       37200.0  
            4027        74800       16000.0  

[4030 rows x 10 columns]

In [19]:
#自身のデータ（customer_id）から過去N件の合計値（price_sum)

# 予約テーブルにprice_avgを追加
reserve_tb['price_sum'] = pd.Series(
  reserve_tb
    # データ行をcustomer_idごとにグループ化
    .groupby('customer_id')
    # customer_idごとにreserve_datetimeでデータを並び替え
    .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True))
    # total_priceのwindow3件にまとめ、その合計値を計算
    .loc[:, 'total_price']
    .rolling(center=False, window=3, min_periods=3).sum()

    # group化を解除すると同時に、customer_idの列を削除
    .reset_index(drop=True)
)

In [22]:
reserve_tb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4030 entries, 0 to 4029
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   reserve_id        4030 non-null   object 
 1   hotel_id          4030 non-null   object 
 2   customer_id       4030 non-null   object 
 3   reserve_datetime  4030 non-null   object 
 4   checkin_date      4030 non-null   object 
 5   checkin_time      4030 non-null   object 
 6   checkout_date     4030 non-null   object 
 7   people_num        4030 non-null   int64  
 8   total_price       4030 non-null   int64  
 9   price_sum         4028 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 315.0+ KB


In [23]:
#自身のデータ（customer_id）から過去N件の平均　自身の過去のデータが3件に満たない場合は、満たない回数内で平均、０回の場合は値なし
# reserve_tbに新たな列としてprice_avgを追加
reserve_tb['price_avg'] = pd.Series(
  reserve_tb

    # データ行をcustomer_idごとにグループ化
    .groupby('customer_id')

    # customer_idごとにreserve_datetimeでデータを並び替え
    .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True))

    # total_priceのwindow3件にまとめ、その平均値を計算
    # min_periodsを1に設定し、1件以上あった場合には計算するよう設定
    ['total_price'].rolling(center=False, window=3, min_periods=1).mean()

    # group化を解除すると同時に、customer_idの列を削除
    .reset_index(drop=True)
)

# customer_idごとにprice_avgを1行下にずらす
reserve_tb['price_avg'] = \
  reserve_tb.groupby('customer_id')['price_avg'].shift(periods=1)


In [27]:
reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN,97200.000000
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0,58900.000000
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0,50466.666667
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0,82866.666667
...,...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,217800.0,139066.666667
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,159000.0,72600.000000
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,112600.0,53000.000000
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,95000.0,NaN


In [28]:
#過去N日間の合計値
import pandas.tseries.offsets as offsets
import operator

# 日時の計算に利用するため、データ型を文字列から日付型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb['reserve_datetime'] = \
  pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# reserve_datetimeの日付を確認せずに、同じcustomer_idのデータ行同士をすべて結合
sum_table = pd.merge(
	reserve_tb[['reserve_id', 'customer_id', 'reserve_datetime']],
  reserve_tb[['customer_id', 'reserve_datetime', 'total_price']]
            .rename(columns={'reserve_datetime': 'reserve_datetime_before'}),
  on='customer_id')

# checkinの日付を比較して、90日以内のデータが結合されているデータ行のみ抽出
# operatorのand_関数を利用して、複合条件を設定
# reserve_idごとにtotal_priceの合計値を計算
# (日付のデータ型については、「第10章 日時型」で詳しく解説)
sum_table = sum_table[operator.and_(
  sum_table['reserve_datetime'] > sum_table['reserve_datetime_before'],
  sum_table['reserve_datetime'] + offsets.Day(-90) <= sum_table['reserve_datetime_before']
)].groupby('reserve_id')['total_price'].sum().reset_index()

# 列名を設定
sum_table.columns = ['reserve_id', 'total_price_sum']

# 計算した合計値を結合し、元のテーブルに情報を付与
# 合計値が存在しないレコードの合計値の値を、fillnaを利用して0に変更
pd.merge(reserve_tb, sum_table, on='reserve_id', how='left').fillna(0)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum,price_avg,total_price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.0,0.000000,0.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.0,97200.000000,0.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0,58900.000000,20600.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0,50466.666667,0.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0,82866.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,217800.0,139066.666667,0.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,159000.0,72600.000000,0.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,112600.0,53000.000000,0.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,95000.0,0.000000,0.0
